In [1]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [2]:
def l2_norm(p, p_ref):
    #Compute and returns the relative L2-norm of the difference between a solution p and a reference solution p_ref
    l2_diff = (numpy.sqrt(numpy.sum((p - p_ref)**2)) / numpy.sqrt(numpy.sum(p_ref**2)))
    return l2_diff

In [3]:
def poisson_2d_jacobi(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    #Solve the 2D poisson equation for a given forcing term using Jacobi relaxation method
    p = p0.copy()
    conv = []
    diff = rtol + 1.0
    ite = 0
    while diff > rtol and ite < maxiter:
        pn = p.copy()
        p[1:-1,1:-1] = (((pn[1:-1,:-2] + pn[1:-1,2:]) * dy**2 + (pn[:-2,1:-1] + pn[2:,1:-1]) * dx**2 - b[1:-1,1:-1] *dx**2 * dy**2) / (2.0 * (dx**2 + dy**2)))
        #Dirichlet boundary conditions at automatically enforced
        #Compute and record the relative L2-norm of the difference
        diff = l2_norm(p,pn)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [4]:
def poisson_solution(x, y, Lx, Ly):
    #Compute and returns the analytical solution of the Poisson equation on a given two-dimensional Cartesian grid
    X, Y = numpy.meshgrid(x, y)
    p = numpy.sin(numpy.pi * X / Lx) * numpy.cos(numpy.pi * Y / Ly)
    return p

In [5]:
#Set paramaters
nx = 101
ny = 101
xmin, xmax = 0.0, 1.0
ymin, ymax = -0.5, 0.5
Lx = xmax - xmin
Ly = ymax - ymin
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)

#Create the gridline locations and the mesh grid
x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

#Create the source term
b = (-2.0 * (numpy.pi / Lx) * (numpy.pi / Ly) * numpy.sin(numpy.pi * X / Lx) * numpy.cos(numpy.pi * Y / Ly))

#Set the initial conditions
p0 = numpy.zeros((ny, nx))

#Compute the analytical solution
p_exact = poisson_solution(x, y, Lx, Ly)

In [6]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    #Solves the 2D poisson equation on a uniform grid, with the same grid spacing in both directions, for a given forcing term using the method of steepest descent
    #The function assumes Dirichlet boundary conditions with value zero. The exit criterion of the solver is based on the relative L2-norm of the solution difference between two consecutive iterations
    def A(p):
        #Apply the Laplacian operator to p
        return (-4.0 * p[1:-1,1:-1] + p[1:-1,:-2] + p[1:-1,2:] + p[:-2,1:-1] +p[2:,1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p) #initial residual
    Ar = numpy.zeros_like(p) #to store the mat-vec multiplication
    conv = []
    diff = rtol + 1
    ite = 0
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        #Compute the residual
        r[1:-1,1:-1] = b[1:-1,1:-1] -A(p)
        #Compute the Laplacian of the residual
        Ar[1:-1,1:-1] = A(r)
        #Compute the step size
        alpha = numpy.sum(r * r) / numpy.sum(r * Ar)
        #Update the solution
        p = pk + alpha * r
        #Dirichlet boundary conditions are automatically enforced
        #Compute the relative L2-norm of the difference
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p , ite, conv

In [7]:
#Compute the solution using the method of steepest descent
p, ites, conv_sd = poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-10)
print('Method of steepest descent: {} iterations '.format(ites) + 'to reach a relative difference of {}'.format(conv_sd[-1]))

Method of steepest descent: 2 iterations to reach a relative difference of 1.3307695446303778e-16


C:\Users\52679\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
#Compute the relative L2-norm of the error
l2_norm(p, p_exact)

8.225076220929745e-05

In [17]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                  maxiter=20000, rtol=1e-6):
    """
    Solves the 2D Poisson equation on a uniform grid,
    with the same grid spacing in both directions,
    for a given forcing term
    using the method of conjugate gradients.
    
    The function assumes Dirichlet boundary conditions with value zero.
    The exit criterion of the solver is based on the relative L2-norm
    of the solution difference between two consecutive iterations.

    Parameters
    ----------
    p0 : numpy.ndarray
        The initial solution as a 2D array of floats.
    b : numpy.ndarray
        The forcing term as a 2D array of floats.
    dx : float
        Grid spacing in the x direction.
    dy : float
        Grid spacing in the y direction.
    maxiter : integer, optional
        Maximum number of iterations to perform;
        default: 20000.
    rtol : float, optional
        Relative tolerance for convergence;
        default: 1e-6.

    Returns
    -------
    p : numpy.ndarray
        The solution after relaxation as a 2D array of floats.
    ite : integer
        The number of iterations performed.
    conv : list
        The convergence history as a list of floats.
    """
    def A(p):
        # Apply the Laplacian operator to p.
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ad = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ite = 0  # iteration index
    # Compute the initial residual.
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    # Set the initial search direction to be the residual.
    d = r.copy()
    while diff > rtol and ite < maxiter:
        pk = p.copy()
        rk = r.copy()
        # Compute the Laplacian of the search direction.
        Ad[1:-1, 1:-1] = A(d)
        # Compute the step size.
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
        # Update the solution.
        p = pk + alpha * d
        # Update the residual.
        r = rk - alpha * Ad
        # Update the search direction.
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        d = r + beta * d
        p = d
        # Dirichlet boundary conditions are automatically enforced.
        # Compute the relative L2-norm of the difference.
        diff = l2_norm(p, pk)
        conv.append(diff)
        ite += 1
    return p, ite, conv

In [21]:
#Compute the solution using the method of conjugate gradients
p, ites, conv_cg = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                                 maxiter=40000,
                                                 rtol=1e-10)
print('Method of conjugate gradients: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_cg[-1]))

C:\Users\52679\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Method of conjugate gradients: 3313 iterations to reach a relative difference of 0.0


In [19]:
#Compute the relative L2-norm of the error
l2_norm(p,p_exact)

1.0

In [20]:
p, ites, conv_jacobi = poisson_2d_jacobi(p0, b, dx, dy,
                                         maxiter=40000,
                                         rtol=1e-10)
print('Jacobi relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_jacobi[-1]))

C:\Users\52679\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Jacobi relaxation: 31227 iterations to reach a relative difference of 9.997923503623598e-11


In [22]:
b = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) +
     numpy.sin(6.0 * numpy.pi * X / Lx) *
     numpy.sin(6.0 * numpy.pi * Y / Ly))

In [23]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b, dx, dy,
                                  maxiter=maxiter, rtol=rtol)
print('Jacobi relaxation: {} iterations'.format(ites))
p, ites, conv = poisson_2d_steepest_descent(p0, b, dx, dy,
                                            maxiter=maxiter,
                                            rtol=rtol)
print('Method of steepest descent: {} iterations'.format(ites))
p, ites, conv = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                              maxiter=maxiter,
                                              rtol=rtol)
print('Method of conjugate gradients: {} iterations'.format(ites))

C:\Users\52679\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


Jacobi relaxation: 31226 iterations
Method of steepest descent: 27059 iterations
Method of conjugate gradients: 3283 iterations
